In [1]:
import numpy as np
import json
import os
import tensorflow as tf
import onnx

In [2]:
# Save tensor as binary file
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


In [3]:
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    intermediate_outputs = {}  # To store outputs for sequential execution

    # Run input through the model layer by layer
    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor to file
        save_tensor_as_bin(x, f"{save_dir}/input/{layer_name}_input.bin")

        # Save weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                save_tensor_as_bin(weight.numpy(), f"{save_dir}/weights/{layer_name}_{weight_name}.bin")

        # Compute layer output
        x = layer(x)

        # Save output tensor to file
        save_tensor_as_bin(x, f"{save_dir}/output/{layer_name}_output.bin")

        # Save layer attributes
        attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
        }
        if hasattr(layer, 'kernel_size'):
            attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            attributes["activation"] = layer.activation.__name__

        save_json(attributes, f"{save_dir}/reference/{layer_name}_attributes.json")

        # Store intermediate outputs for sequential execution
        intermediate_outputs[layer_name] = x.numpy()

    return intermediate_outputs


In [4]:
model = tf.keras.models.load_model("/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/CPU_trained_models/cifar10_model_4.h5")

In [5]:
input_tensor = tf.random.normal([1, 32, 32, 3])  # Random input for CIFAR-10
layer_wise_dump_tf(model, input_tensor, save_dir="data")

{'conv2d': array([[[[0.00000000e+00, 1.48461103e+00, 0.00000000e+00, ...,
           1.75928548e-01, 4.52473551e-01, 2.43212178e-01],
          [4.43924218e-01, 1.44059047e-01, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [6.22040689e-01, 0.00000000e+00, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 7.29394797e-03],
          ...,
          [0.00000000e+00, 3.70511651e-01, 0.00000000e+00, ...,
           1.03078961e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 1.06160730e-01],
          [0.00000000e+00, 0.00000000e+00, 4.16199952e-01, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
         [[1.01239777e+00, 0.00000000e+00, 1.98551193e-01, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 8.23761880e-01, 0.00000000e+00, ...,
           5.38316846e-01, 1.23048849e-0

In [19]:
import os
import json
import numpy as np

def generate_layer_config(model, save_dir, config_file_path):
    # Build model or run a dummy forward pass to infer shapes
    if not model.built:
        # Infer the input shape from the model's first layer
        input_shape = model.input_shape if hasattr(model, 'input_shape') else None
        if input_shape:
            model.build(input_shape)
        else:
            # Use dummy input if input_shape is unknown
            dummy_input = np.random.random((1, *model.layers[0].input_shape[1:]))
            model(dummy_input)
    
    layer_configs = []  # List to store layer-wise configurations

    for layer in model.layers:
        layer_name = layer.name

        # Construct layer configuration
        layer_config = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,  # Correct way to get the class name
            "input_file_path": f"{save_dir}/input/{layer_name}_input.bin",
            "output_file_path": f"{save_dir}/output/{layer_name}_output.bin",
            "weights_file_paths": [],  # To be filled if the layer has weights
            "attributes": {
                "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
                "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            }
        }

        # Add weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_file_path = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                layer_config["weights_file_paths"].append(weight_file_path)

        # Add other layer-specific attributes if available
        if hasattr(layer, 'kernel_size'):
            layer_config["attributes"]["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_config["attributes"]["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_config["attributes"]["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_config["attributes"]["activation"] = layer.activation.__name__

        # Append this layer's config to the list
        layer_configs.append(layer_config)

    # Save all configurations to a JSON file
    with open(config_file_path, 'w') as config_file:
        json.dump({"layers": layer_configs}, config_file, indent=4)

    print(f"Configuration file saved to {config_file_path}")


In [20]:
# Assuming the `data` folder is in the current directory
data_folder = "/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/data"
output_json_file = "/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/configs/json/model_config.json"
model = tf.keras.models.load_model("/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/CPU_trained_models/cifar10_model_4.h5")

generate_layer_config(model, data_folder, output_json_file)


Configuration file saved to /Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/configs/json/model_config.json
